In [0]:
import pandas as pd
pd.__version__

'1.0.3'

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
ted = pd.read_csv('https://raw.githubusercontent.com/justmarkham/pycon-2019-tutorial/master/ted.csv')

In [0]:
# each row represents a single talk
ted.head()

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","[{'id': 243, 'hero': 'https://pe.tedcdn.com/im...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520
2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,26,David Pogue,David Pogue: Simplicity sells,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...","[{'id': 1725, 'hero': 'https://pe.tedcdn.com/i...",Technology columnist,"['computers', 'entertainment', 'interface desi...",Simplicity sells,https://www.ted.com/talks/david_pogue_says_sim...,1636292
3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","[{'id': 1041, 'hero': 'https://pe.tedcdn.com/i...",Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,https://www.ted.com/talks/majora_carter_s_tale...,1697550
4,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...","[{'id': 2056, 'hero': 'https://pe.tedcdn.com/i...",Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,https://www.ted.com/talks/hans_rosling_shows_t...,12005869


In [0]:
# row, columns 
ted.shape

(2550, 17)

In [0]:
# object columns are usually strings, but can also be arbitrary Python objects (lists, dictionaries)
ted.dtypes

comments               int64
description           object
duration               int64
event                 object
film_date              int64
languages              int64
main_speaker          object
name                  object
num_speaker            int64
published_date         int64
ratings               object
related_talks         object
speaker_occupation    object
tags                  object
title                 object
url                   object
views                  int64
dtype: object

In [0]:
# count the number of missing values in each column
ted.isna().sum()


comments              0
description           0
duration              0
event                 0
film_date             0
languages             0
main_speaker          0
name                  0
num_speaker           0
published_date        0
ratings               0
related_talks         0
speaker_occupation    6
tags                  0
title                 0
url                   0
views                 0
dtype: int64

In [0]:
# sort by the number of first-level comments, though this is biased in favor of older talks
ted.sort_values('comments').tail()

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views
1787,2673,Our consciousness is a fundamental aspect of o...,1117,TED2014,1395100800,33,David Chalmers,David Chalmers: How do you explain consciousness?,1,1405350484,"[{'id': 25, 'name': 'OK', 'count': 280}, {'id'...","[{'id': 1308, 'hero': 'https://pe.tedcdn.com/i...",Philosopher,"['brain', 'consciousness', 'neuroscience', 'ph...",How do you explain consciousness?,https://www.ted.com/talks/david_chalmers_how_d...,2162764
201,2877,Jill Bolte Taylor got a research opportunity f...,1099,TED2008,1204070400,49,Jill Bolte Taylor,Jill Bolte Taylor: My stroke of insight,1,1205284200,"[{'id': 22, 'name': 'Fascinating', 'count': 14...","[{'id': 184, 'hero': 'https://pe.tedcdn.com/im...",Neuroanatomist,"['biology', 'brain', 'consciousness', 'global ...",My stroke of insight,https://www.ted.com/talks/jill_bolte_taylor_s_...,21190883
644,3356,"Questions of good and evil, right and wrong ar...",1386,TED2010,1265846400,39,Sam Harris,Sam Harris: Science can answer moral questions,1,1269249180,"[{'id': 8, 'name': 'Informative', 'count': 923...","[{'id': 666, 'hero': 'https://pe.tedcdn.com/im...","Neuroscientist, philosopher","['culture', 'evolutionary psychology', 'global...",Science can answer moral questions,https://www.ted.com/talks/sam_harris_science_c...,3433437
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110
96,6404,Richard Dawkins urges all atheists to openly s...,1750,TED2002,1012608000,42,Richard Dawkins,Richard Dawkins: Militant atheism,1,1176689220,"[{'id': 3, 'name': 'Courageous', 'count': 3236...","[{'id': 86, 'hero': 'https://pe.tedcdn.com/ima...",Evolutionary biologist,"['God', 'atheism', 'culture', 'religion', 'sci...",Militant atheism,https://www.ted.com/talks/richard_dawkins_on_m...,4374792


In [0]:
# correct for this bias by calculating the number of comments per view
ted["comment_per_view"] = ted.comments/ted.views

In [0]:
# correct for this bias by calculating the number of comments per view
ted['comments_per_view'] = ted.comments / ted.views

In [87]:
# interpretation: 1 out of every 450 people leave a comment
ted.sort_values('view_per_comment').head()

KeyError: ignored

In [0]:
# correct for this bias by calculating the number of comments per view
ted['comments_per_view'] = ted.comments / ted.views

In [0]:
# interpretation: for every view of the same-sex marriage talk, there are 0.002 comments
ted.sort_values('comments_per_view').tail()

In [0]:
# make this more interpretable by inverting the calculation
ted['views_per_comment'] = ted.views / ted.comments

In [0]:
# interpretation: 1 out of every 450 people leave a comment
ted.sort_values('views_per_comment').head()

In [0]:
# line plot is not appropriate here (use it to measure something over time)
ted.comments.plot()

In [0]:
# histogram shows the frequency distribution of a single numeric variable
ted.comments.plot(kind='hist')

In [0]:
# modify the plot to be more informative
ted[ted.comments < 1000].comments.plot(kind='hist')

In [0]:
# check how many observations we removed from the plot
ted[ted.comments >= 1000].shape

In [0]:
# can also write this using the query method
ted.query('comments < 1000').comments.plot(kind='hist')

In [0]:
# can also write this using the loc accessor
ted.loc[ted.comments < 1000, 'comments'].plot(kind='hist')

In [0]:
# increase the number of bins to see more detail
ted.loc[ted.comments < 1000, 'comments'].plot(kind='hist', bins=20)

In [0]:
# boxplot can also show distributions, but it's far less useful for concentrated distributions because of outliers
ted.loc[ted.comments < 1000, 'comments'].plot(kind='box')

In [0]:
# event column does not always include the year
ted.event.sample(10)

In [0]:
# dataset documentation for film_date says "Unix timestamp of the filming"
ted.film_date.head()

In [0]:
# results don't look right
pd.to_datetime(ted.film_date).head()

In [0]:
# now the results look right
pd.to_datetime(ted.film_date, unit='s').head()

In [0]:
ted['film_datetime'] = pd.to_datetime(ted.film_date, unit='s')

In [0]:
# verify that event name matches film_datetime for a random sample
ted[['event', 'film_datetime']].sample(5)

In [0]:
# new column uses the datetime data type (this was an automatic conversion)
ted.dtypes

In [0]:
# datetime columns have convenient attributes under the dt namespace
ted.film_datetime.dt.year.head()

In [0]:
# similar to string methods under the str namespace
ted.event.str.lower().head()

In [0]:
# count the number of talks each year using value_counts()
ted.film_datetime.dt.year.value_counts()

In [0]:
# points are plotted and connected in the order you give them to pandas
ted.film_datetime.dt.year.value_counts().plot()

In [0]:
# need to sort the index before plotting
ted.film_datetime.dt.year.value_counts().sort_index().plot()

In [0]:
# we only have partial data for 2017
ted.film_datetime.max()

In [0]:
# count the number of talks (great if you value variety, but they may not be great talks)
ted.event.value_counts().head()

In [0]:
# count the number of talks (great if you value variety, but they may not be great talks)
ted.event.value_counts().head()

In [0]:
# use views as a proxy for "quality of talk"
ted.groupby('event').views.mean().head()

In [0]:
# find the largest values, but we don't know how many talks are being averaged
ted.groupby('event').views.mean().sort_values().tail()

In [0]:
# show the number of talks along with the mean (events with the highest means had only 1 or 2 talks)
ted.groupby('event').views.agg(['count', 'mean']).sort_values('mean').tail()

In [0]:
# calculate the total views per event
ted.groupby('event').views.agg(['count', 'mean', 'sum']).sort_values('sum').tail()

In [0]:
# previously, users could tag talks on the TED website (funny, inspiring, confusing, etc.)
ted.ratings.head()

In [0]:
# two ways to examine the ratings data for the first talk
ted.loc[0, 'ratings']
ted.ratings[0]

In [0]:
# this is a string not a list
type(ted.ratings[0])

In [0]:
# convert this into something useful using Python's ast module (Abstract Syntax Tree)
import ast

In [0]:
# convert this into something useful using Python's ast module (Abstract Syntax Tree)
import ast

In [0]:
# if you have a string representation of something, you can retrieve what it actually represents
type(ast.literal_eval('[1, 2, 3]'))

In [0]:
# unpack the ratings data for the first talk
ast.literal_eval(ted.ratings[0])

In [0]:
# now we have a list (of dictionaries)
type(ast.literal_eval(ted.ratings[0]))

In [0]:
# define a function to convert an element in the ratings Series from string to list
def str_to_list(ratings_str):
    return ast.literal_eval(ratings_str)

In [0]:
# test the function
str_to_list(ted.ratings[0])

In [0]:
 # Series apply method applies a function to every element in a Series and returns a Series
ted.ratings.apply(str_to_list).head()

In [0]:
# lambda is a shorter alternative
ted.ratings.apply(lambda x: ast.literal_eval(x)).head()

In [0]:
# lambda is a shorter alternative
ted.ratings.apply(lambda x: ast.literal_eval(x)).head()

In [0]:
ted['ratings_list'] = ted.ratings.apply(lambda x: ast.literal_eval(x))

In [0]:
# check that the new Series looks as expected
ted.ratings_list[0]

In [0]:
# each element in the Series is a list
type(ted.ratings_list[0])

In [0]:
# data type of the new Series is object
ted.ratings_list.dtype

In [0]:
# object is not just for strings
ted.dtypes